### Checklist for submission

It is extremely important to make sure that:

1. Everything runs as expected (no bugs when running cells);
2. The output from each cell corresponds to its code (don't change any cell's contents without rerunning it afterwards);
3. All outputs are present (don't delete any of the outputs);
4. Fill in all the places that say `YOUR CODE HERE`, or "**Your answer:** (fill in here)".
5. You **ONLY** change the parts of the code we asked you to, nowhere else (change only the coding parts saying `# YOUR CODE HERE`, nothing else);
6. Don't add any new cells to this notebook;
7. Fill in your group number and the full names of the members in the cell below;
8. Make sure that you are not running an old version of IPython (we provide you with a cell that checks this, make sure you can run it without errors).

Failing to meet any of these requirements might lead to either a subtraction of POEs (at best) or a request for resubmission (at worst).

We advise you the following steps before submission for ensuring that requirements 1, 2, and 3 are always met: **Restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All). This might require a bit of time, so plan ahead for this (and possibly use Google Cloud's GPU in HA1 and HA2 for this step). Finally press the "Save and Checkout" button before handing in, to make sure that all your changes are saved to this .ipynb file.

---

Group number and member names:

In [1]:
GROUP = "63"
NAME1 = "Simon Persson"
NAME2 = "Guo Wu"

Make sure you can run the following cell without errors.

In [2]:
import IPython
assert IPython.version_info[0] >= 3, "Your version of IPython is too old, please update it."

---

# HA2 - Recurrent Neural Networks

Welcome to the second group home assignment!  
The purpose of this assigment is to give you practical knowledge in how to implement recurrence in a neural network.  

Every day you are exposed to sequences of data. For example text, video streams, audio, financial time series and medical sensors. Recurrence is therefore an important topic in the field of Machine Learning because it has the potential to solve real-world problems including the types of data above.  

In this assignment, you will learn to:
 - Preprocess text data sets
 - Implement a vanilla RNN cell using only numpy
 - Text generation
 - Build a recurrent neural network with LSTM using Keras  
 - Neural machine translation

**NOTE:** Task 1 (Shallow vanilla RNN) and Task 2 (Neural machine translation), are independent from each other. Task 2 asks you to train a NMT, which takes a while (specially without a GPU), so it might be efficient to start with task 2 and leave it running in the background while you solve task 1.

Table of Contents:  
 [1 Shallow vanilla RNN](#1)  
   [1.1 Preprocessing](#1.1)  
     [1.1.1 Loading dataset](#1.1.1)  
     [1.1.2 One-hot representations](#1.1.2)  
   [1.2 RNN cell class](#1.2)  
   [1.3 Training the RNN](#1.3)  
 [2 Neural machine translation](#2)  
   [2.1 Pre-processing](#2.1)  
     [2.1.1 Loading and inspecting dataset](#2.1.1)  
     [2.1.2 Cleaning the dataset](#2.1.2)  
     [2.1.3 Restricting sentence length and shuffling the data set](#2.1.3)  
     [2.1.4 Word-to-index and index-to-word conversions](#2.1.4)  
     [2.1.5 Padding](#2.1.5)  
     [2.1.6 One-hot labels](#2.1.6)  
   [2.2 Implementing a sequence-to-sequence model](#2.2)  
     [2.2.1 Defining the architecture](#2.2.1)  
     [2.2.2 Training the model](#2.2.2)  
     [2.2.3 Evalutation](#2.2.3)  
     [2.2.4 Testing](#2.2.4)  
     
**NOTE**: The tests available are not exhaustive, meaning that if you pass a test you have avoided the most common mistakes, but it is still not guaranteed that you solution is 100% correct.

## 1 Shallow vanilla RNN <a class="anchor" id="1"></a>
In the first part of this assignment, you will implement a recurrent neural network from scratch without using any framework. You will train this network to predict the next character of a text, which will result in a network that can generate new sentences.  

Start by importing dependencies below  

In [3]:
import numpy as np
import copy
import matplotlib.pyplot as plt
from utils.tests.ha2Tests import *

### 1.1 Preprocessing <a class="anchor" id="1.1"></a>
#### 1.1.1 Loading data set <a class="anchor" id="1.1.1"></a>
The text corpus to train your RNN on is going to be the book The metamorphosis by Franz Kafka. Run the cell below to load the text.

In [4]:
data = open('./utils/kafka.txt', 'r', encoding="utf-8").read()

chars = list(set(data)) 
data_size, vocab_size = len(data), len(chars)
print('data has %d chars, %d unique' % (data_size, vocab_size))

data has 118560 chars, 62 unique


#### 1.1.2 One-hot representations <a class="anchor" id="1.1.2"></a>
`data` is now a string, containing the contents of the book, with 80 unique characters. As usual, converting the labels (that is also the data in our case) into one-hot vectors is a good way for creating unbiased labels.  

Below is a variable `one_hot_vectors` defined that maps every character in `chars` to a unique one-hot vector  

In [5]:
one_hot_vectors = np.eye(len(chars))
char_to_onehot = { val: one_hot_vectors[key,:].reshape(1,-1) for key, val in enumerate(chars) }

### 1.2 RNN cell class <a class="anchor" id="1.2"></a>

Now you will implement the RNN class. For your convenience, this task is split into 4 subtasks, one for each method of the class. These methods will be later linked to a class definition (if you're curious, scroll down to 1.2.5).

Instead of implementing a neural network of arbitrary length like in **IHA1**, you are only going to implement a single RNN cell. At each time-step, the RNN cell will have one input, $\mathbf{x}_t$ , one hidden layer with a hidden state $\mathbf{h}_t$, and one output $\mathbf{y}_t$:

![Simple RNN cell](utils/images/RNN-cell.png)

The output at time-step $t$, referred to as $\mathbf{y}_t$, is computed according to the following equations. Equation 1 and 2 together define the next hidden state $\mathbf{h}_t$, and Equation 3 defines the unnormalized probability output $\mathbf{z}_t$.

$$
\mathbf{z}_t =  \mathbf{x}_t \mathbf{W}_{xh} + \mathbf{h}_{t-1} \mathbf{W}_{hh} + \mathbf{b}_h, \tag{1}
$$
$$
\mathbf{h}_t = \tanh (\mathbf{z}_t), \tag{2}
$$
$$
\mathbf{y}_t =  \mathbf{h}_t \mathbf{W}_{hy} + \mathbf{b}_y, \tag{3}
$$

where $\mathbf{W}_{xh}$, $\mathbf{W}_{hh}$, $\mathbf{W}_{hy}$, $\mathbf{b}_{h}$, and $\mathbf{b}_{y}$ are the parameters of the RNN cell.

Calculating $\mathbf{h}_{30}$ means that you need to need to have calculated every earlier hidden state $\{\mathbf{h}_t | t \in \{0, 1, ..., 29\}\}$. This can easier be understood by always imagining an unrolled RNN cell:
![Simple RNN cell unrolled](utils/images/RNN-cell-unrolled.png)
  

Since it is a multiclass classification problem, the softmax activation function is going to be used to normalize the output of the RNN cell $\mathbf{y}_t$, defined as

$$
\mathbf{p}_t = softmax(\mathbf{y}_t) = \frac{e^{\mathbf{y}_t - max(\mathbf{y_t})}}{ \sum^j e^{y_{j_t} - max(\mathbf{y_t})}}. \tag{4}
$$

#### 1.2.1 The initalization method

The first step in creating the RNN class is to create its `__init__` method, where we create all the necessary attributes and initialize them. Complete the `init_routine` function below. Later this will be linked to the `__init__` method in the `SimpleRNN` class, so you're now developing the constructor of that class.

In [6]:
def init_routine(self, input_dim, hidden_dim, output_dim):
    """
    Initialize the weights of the RNN and define a cache with the starting value of the hidden state

    Arguments:
    self - an object of the class SimpleRNN
    input_dim - an integer representing the number of inputs
    hidden_dim - an integer representing the length of the hidden state vector
    output_dim - an integer representing the number of outputs

    Attributes:
    input_dim - attribute initialized with the value of the argument `input_dim`
    cache - a dictionary holding the values of `h_start`, `xs` and `hs` from the previous forward propagation call.
            h_start - the value to initialize the hidden state with when `forward_prop is called. The value of
                      `h_start` should be initialized to a `numpy.ndarray` vector of zeros 
            xs - the list of `xs` used in the last `forward_prop` call. Later used in `backward_prop`. Can be
                 initialized to `None`
            hs - the list of `hs` calculated in the last `forward_prop` call. Later used i `backward_prop`. Can
                 be initialized to `None`
    W_hh - weight matrix of shape (hidden_dim, hidden_dim) and type `numpy.ndarray`. 
           Initialized randomly from a normal distribution of mean zero and stddev 0.01
    W_xh - weight matrix of shape (input_dim, hidden_dim) and type `numpy.ndarray`. 
           Initialized randomly from a normal distribution of mean zero and stddev 0.01
    W_hy - weight matrix of shape (hidden_dim, output_dim) and type `numpy.ndarray`. 
           Initialized randomly from a normal distribution of mean zero and stddev 0.01
    b_h -  bias of shape (1, hidden_dim) and type `numpy.ndarray`. Initialized to all zeros
    b_y -  bias of shape (1, output_dim) and type `numpy.ndarray`. Initialized to all zeros
    """

    self.input_dim = input_dim
    self.hidden_dim = hidden_dim
    self.output_dim = output_dim
    self.cache = {
        'h_start': np.zeros((1, hidden_dim))
    }
    
    # YOUR CODE HERE
    self.cache['xs'] = None
    self.cache['hs'] = None
    self.W_hh = np.random.normal(loc=0,scale=0.01,size=(hidden_dim, hidden_dim))
    self.W_xh = np.random.normal(loc=0,scale=0.01,size=(input_dim, hidden_dim))
    self.W_hy = np.random.normal(loc=0,scale=0.01,size=(hidden_dim, output_dim))
    self.b_h = np.zeros((1,hidden_dim))
    self.b_y = np.zeros((1,output_dim))

The following cell tests if your implementation is correct.

In [7]:
test_init_routine(init_routine)

test init passed


#### 1.2.2 Forward propagation

Now that we have an initialization method for declaring objects of our class, we need a method for performing the forward propagation step. Complete the `forward_prop_routine` function. This will later be linked to the `forward_prop` routine of the `SimpleRNN` class. 

In [8]:
def forward_prop_routine(self, xs, reset_h=False):
        """
        Performs forward propagation of the input `xs` in sequential order, and then returns all predictions `ys_pred`
        The steps are: [Input x] -> [Hidden h] -> [Output unnormalized z] -> [Prediction probabilities y]

        Arguments:
        xs - a python list of one-hot characters to predict the next character from,
             Has a shape of list([1,OUTPUT_DIM]) and length `len(xs)`
        reset_h - boolean value, whether or not the hidden state should be reset

        Returns:
        ys_pred - a python list of prediction probabilities for each character in `xs`. There are `len(xs)`
                  elements in ys_pred, each element `i` of shape (1, output_dim) and type `numpy.ndarray`
                  that contains the probabilities of what the next character xs[i+1] can be

        Example:
        xs - [np.array([0,0,1]), np.array([0,1,0]), np.array([1,0,0])]
        ys_pred (ground truth answer) - [np.array([0,1,0]), np.array([1,0,0]), np.array([...])]
        """
        # initialize the list of predictions
        ys_pred = [0] * len(xs)

        # load the last hidden state of the previous batch
        if reset_h:
            self.cache['h_start'] = np.zeros((1, self.W_hh.shape[0]))
        hs = {-1: self.cache['h_start']}

        # YOUR CODE HERE      
        def softmax(yt):
            max = np.max(yt)
            numerator = np.exp(yt-max)
            denominator = np.sum(numerator)
            return np.divide(numerator,denominator)     
        
        
        for i in range(len(xs)):
            z = np.dot(xs[i],self.W_xh) + np.dot(hs[i-1],self.W_hh) + self.b_h
            hs[i] = np.tanh(z)
            ys_pred[i] = np.dot(hs[i],self.W_hy) + self.b_y
            ys_pred[i] = softmax(ys_pred[i]) 
        
        
        
        # save list of hidden state in cache to use later in backprop
        self.cache = {
            'hs': hs,
            'xs': xs,
            'h_start': hs[len(xs) - 1]
        }

        return ys_pred

The following cell tests if your implementation is correct (it assumes you correctly solved task 1.2.1):

In [9]:
test_forward_prop_routine(init_routine, forward_prop_routine)

test passed, dimensions are correct


#### 1.2.3 Backward propagation

Now that you developed methods for initialization and forward propagation, it's time to implement the backward propagation algorithm as a method for our class.

The cross-entropy loss will be used for this task, and just like in **IHA1**, backward pass of the softmax activation function and the cross-entropy loss can be combined into a simple formula: $\mathbf{\hat{p_t}} - \mathbf{p_t}$, where $\mathbf{\hat{p_t}}$ is the predicted output vector for time step $t$ and $\mathbf{p_t}$ is the ground truth one-hot vector for time step $t$.
The full collection of backward pass formulas that are required for implementing `backward_prop_routine` are shown in Equations 5-12:  

$$
\frac{d\mathcal{L}_t}{d \mathbf{y}_t}=\hat{\mathbf{p}}_t-\mathbf{p}_t \tag{5} ~,~ \text{ for } t=1,\dots,N, 
$$

$$
\frac{d\mathcal{L}}{d\mathbf{W}_{hy}}=\sum_{t=1}^N\mathbf{h}_t^T\frac{d\mathcal{L}_t}{d \mathbf{y}_t}, \tag{6}
$$

$$
\frac{d\mathcal{L}}{d\mathbf{b}_{y}}=\sum_{t=1}^N\frac{d\mathcal{L}_t}{d \mathbf{y}_t},   \tag{7}
$$

$$
\frac{d\mathcal{L}}{d\mathbf{h}_t}=\frac{d \mathcal{L}_t}{d \mathbf{y}_t}\mathbf{W}_{hy}^T+(1-\mathbf{h}_{t+1}^2)\odot\frac{d\mathcal{L}}{d\mathbf{h}_{t+1}}\mathbf{W}_{hh}^T   \tag{8.1} ~,~ \text{ for }t=1,\dots,N-1,
$$

$$
\frac{d\mathcal{L}}{d\mathbf{h}_N}=\frac{d \mathcal{L}_t}{d \mathbf{y}_N}\mathbf{W}_{hy}^T,   \tag{8.2}
$$

$$
\frac{d\mathcal{L}}{d\mathbf{z}_t}=(1-\mathbf{h}_t^2)\odot\frac{d\mathcal{L}}{d\mathbf{h}_t}   ~,~ \text{ for } t=1,\dots,N,\tag{9}
$$

$$
\frac{d\mathcal{L}}{d\mathbf{W}_{xh}}=\sum_{t=1}^N\mathbf{x}_t^T\frac{d\mathcal{L}}{d\mathbf{z}_t},   \tag{10}
$$

$$
\frac{d\mathcal{L}}{d\mathbf{W}_{hh}}=\sum_{t=1}^N\mathbf{h}_{t-1}^T\frac{d\mathcal{L}}{d\mathbf{z}_t},   \tag{11}
$$

$$
\frac{d\mathcal{L}}{d\mathbf{b}_h}=\sum_{t=1}^N\frac{d\mathcal{L}}{d\mathbf{z}_t},   \tag{12}
$$
where $\odot$ is the <a href="https://en.wikipedia.org/wiki/Hadamard_product_(matrices)">hadamard product / elementwise multiplication</a>.  

Complete the `backward_prop_routine` function below. This will later be linked to the `backward_prop` method of the `SimpleRNN` class.

In [10]:
def backward_prop_routine(self, ys, ys_pred):
    """
    Performs backward propagation, calculating the gradients of every trainable weight. The gradients should
    be clipped into the interval [-5,5]

    Arguments:
    ys - a python list of true labels (one-hot vectors of type `numpy.ndarray` for every character to predict)
         has a shape list( (1,OUTPUT_DIM) )
    ys_pred - a python list of predicted labels (one-hot vectors of type `numpy.ndarray` for every character predicted)
         has a shape list( (1,OUTPUT_DIM) )

    Returns:
    gradients - a dictionary of the gradients of every trainable weight. The keys are the weight variable names
                and the values are the actual gradient value
    """
    
    # extract from cache
    hs = self.cache['hs']
    xs = self.cache['xs']
    
    # YOUR CODE HERE
    N = len(ys)    

    Lt_yt = np.zeros((N,1,self.output_dim))
    dW_hy = np.zeros((self.W_hy.shape))
    db_y = np.zeros((self.b_y.shape))
    dW_xh = np.zeros((self.W_xh.shape))
    dW_hh = np.zeros((self.W_hh.shape))
    db_h = np.zeros((self.b_h.shape))
    dh_t = np.zeros((N,1,self.hidden_dim))
    dz_t = np.zeros((N,1,self.hidden_dim))
    


    for t in range(N):
        Lt_yt[t] = ys_pred[t] - ys[t]
        dW_hy += np.dot(hs[t].T, Lt_yt[t])
        db_y += Lt_yt[t]
        
    for t in range(N-1,-1,-1):
        if t == N-1:
            dh_t[t] = np.dot(Lt_yt[t], self.W_hy.T)
        else:
            dh_t[t] = np.dot(Lt_yt[t],self.W_hy.T)+np.dot((1-np.power(hs[t+1],2))*(dh_t[t+1]),self.W_hh.T)
    for t in range(N):
        dz_t[t] = (1 - np.power(hs[t],2)) * dh_t[t]
        dW_xh += np.dot(xs[t].T, dz_t[t])
        dW_hh += np.dot(hs[t-1].T, dz_t[t])
        db_h += dz_t[t]
            
            
            
            

    # Clip gradients to prevent explosion
    for dparam in [dW_xh, dW_hh, dW_hy, db_h, db_y]:
        np.clip(dparam, -5, 5, out=dparam)

    # save gradients into dict and return
    gradients = {
        'W_xh': dW_xh, 
        'W_hh': dW_hh, 
        'W_hy': dW_hy, 
        'b_h': db_h, 
        'b_y': db_y
    }

    return gradients

The following cell tests if your implementation is correct (it assumes you correctly solved tasks 1.2.1 and 1.2.2):

In [11]:
test_backward_prop_routine(init_routine, forward_prop_routine, backward_prop_routine)

Your BPTT implementation is correct.


#### 1.2.4 Applying the gradients

Finally, we need a method that updates the paramters of the RNN cell, given the computed gradients of the loss with respect to each one of them. Complete the `apply_gradients_routine` function below. This will later be linked to the `apply_gradients` method of the `SimpleRNN` class.

In [274]:
def apply_gradients_routine(self, gradients, learning_rate):
    """ Performs the weight update procedure. Updates every weight with its corresponding gradient 
    found in the `gradients` input dictionary

    Arguments:
    gradients - dictionary containing (key, value) pairs, where the key is a weight variable name of
                the `SimpleRNN` class and the value is the gradient to apply to the matching weight
                Example: {'W_xh':0.04, 'b_h':0.11}
    learning_rate - the learning rate to use for this iteration of weight updates
    """

    # YOUR CODE HERE
    self.W_xh -= gradients['W_xh'] * learning_rate 
    self.W_hh -= gradients['W_hh'] * learning_rate
    self.W_hy -= gradients['W_hy'] * learning_rate
    self.b_h -= gradients['b_h'] * learning_rate
    self.b_y -= gradients['b_y'] * learning_rate

The following cell tests if your implementation is correct (it assumes you correctly solved task 1.2.1):

In [275]:
test_apply_gradients_routine(init_routine, apply_gradients_routine)

test apply gradients passed


#### 1.2.5 Putting everything together
Now that all the parts of the RNN are implemented, we can define the class. The following cell defines the `SimpleRNN` class, linking all the functions you developed in the earlier tasks to corresponding methods. Additionally, it also implements another method, the `sample`, used to generate output from a trained RNN. Note that you don't have to change anything in this cell, just run it after solving the previous tasks.

In [276]:
class SimpleRNN:
    """
    The simple RNN class definition
    Implements the initializations of the weights, the forward propagation from `x` to `y_pred`,
    the backward propagation with respect to each trainable weight and the update weights rule.
    """
    
    # __init__ method is now the initialization_routine function
    __init__ = init_routine
        
    # forward_prop method is the forward_prop_routine function
    forward_prop = forward_prop_routine
    
    # backward_prop method is the backward_prop_routine function
    backward_prop = backward_prop_routine
    
    # apply_gradients method is the apply_gradients_routine function
    apply_gradients = apply_gradients_routine


    def sample(self, seed, n, char_to_onehot, chars):
        """ Given a seed character `seed`, generates a sequence of `n` characters by continuously feeding the output
        character of the RNN at time t as the input for the next time step at time t+1. If the RNN is trained well, 
        this method should be able to generate sentences that resembles some kind of structure. 
        The character should be generated with a probability of the outputs of the network!
        
        Arguments:
        seed - a first character of type str to feed into the SimpleRNN
        n - the length of the character sequence to generate
        char_to_onehot - a dict that maps keys of characters to its one-hot representation. 
                         You created this dict earlier in the lab
        char - a list of chars (vocabulary). You also created this list earlier in the lab
        """
        
        saved_cache = copy.deepcopy(self.cache)
        
        char_list = []
        h = self.cache['h_start']

        # seed one-hot char
        x = char_to_onehot[seed]

        for t in range(n):
            """ not relevant anymore
            # perform forward prop. `forward_prop` is not called because then the hidden state for the
            # training procedure will be overwritten by the hidden state generates by this sampling procedure
            h = np.tanh(np.dot(x, self.W_xh) + np.dot(h, self.W_hh) + self.b_h)
            y = np.dot(h, self.W_hy) + self.b_y
            p = np.exp(y) / np.sum(np.exp(y))
            """
            p = self.forward_prop(x)[0]
            
            # randomly pick a character with respect to the probabilities of the output of the network
            ix = np.random.choice(range(self.input_dim), p=p.ravel())

            x = np.zeros((1, self.input_dim))   
            x[0,ix] = 1

            # save the generated character
            char_list.append(chars[ix])
            
        self.cache = saved_cache

        return ''.join(char_list)

### 1.3 Training the RNN <a class="anchor" id="1.3"></a>
In this section, code has been provided to you to train a `SimpleRNN` cell by predicting the next charcter in sequence.  

The code below has the following properties: 
 * Loop over `data`, using `seq_length` words at a time as a batch to train with
 * When `data` has been fully iterated through, start over again
 * Start over again with `data` in an infinite loop and then interrupt the running cell
 when results from `sample` are good enough.  
 * Perform `forward_prop`, `backward_prop` and  `apply_gradients` with each batch of characters
 * every 1000th iteration:
     * compute the cross-entropy loss of the current batch
     * sample a character sequence of length 200
     * print the loss and the sampled sequence

In [277]:
rnn = SimpleRNN(len(chars), 500, len(chars))

# the number of characters to perform one update of the network with,
# can be thought of as a batch
seq_length = 25

# the learning rate to use
learning_rate = 0.001

while True: # the infinite loop
    
    # iterate over the data set
    for i in range(len(data)//seq_length):
        
        # convert to one-hot
        xs = [char_to_onehot[c] for c in data[i*seq_length:(i+1)*seq_length]]#inputs to the RNN
        ys = [char_to_onehot[c] for c in data[i*seq_length+1:(i+1)*seq_length+1]]#the targets it should be outputting

        # fp, bp and update
        ys_pred = rnn.forward_prop(xs)
        gradients = rnn.backward_prop(ys, ys_pred)
        rnn.apply_gradients(gradients, learning_rate)

        # print loss and sample a sentence every 1000th iteration
        if i%1000==0:
            loss = np.sum([ -1.0 *np.sum(y * np.log(y_pred + 1e-8)) for y, y_pred in zip(ys, ys_pred) ]) # x-entropy
            print('iteration %d, loss: %f' % (i, loss))
            text = rnn.sample('a', 200, char_to_onehot, chars)
            print(text, end='\n\n')

iteration 0, loss: 103.189606
uuYUeç!qbWDçzhQE)-jIoo!A.mFGc;DkT?WLfBç"FPqEwFtçPDçneMh'a!Dk;UodlU";SvbyrAMçMPonAHm;hiTT-j()OHsuMY"kSIC,HWSGdzvg
)Bog:YIumFIYh
s-)snNvçWHJ;BCHDgHP:l(fsmSlxugaQeBH?H.;Chuxr
eCWoeçEo:-v-mgBwy)qtHfCNPIxI

iteration 1000, loss: 78.656703
e tsetvctwerisrclnysplesgsasne 'urh,i eesLh!d nuarcfik otideo.ffa i ooches  tte.cdt,httdh.hshfldyrdg uild ie r t,rirGstgeiltv h rw l tVyuhsrifswo   rr ae cllge,g  kaa,eet,etiot,l thitho nro r,iiygstuT

iteration 2000, loss: 69.727664
fb de awabla s ho bnso er wntudersc oi ths s rsrme dat sosrrer  s aoh(d xoe toyirn hsilidnofeflgh hrhhuaep rhnioc.lcred,sns ah wrwhgetcreege  oser lno  gloarnse t h  ye yrawnodee  uem n ooi ee r oo ir

iteration 3000, loss: 67.898546
on Ghisewncde ooihlmwhub mo ieos naY,rod rwmmuhat haallgrdiy sotstnNda enr gu aie t aha cgsgaae nosasag ffdem was nonolg ,tiua dathafrpgpoeaherenisBr her hlkoaC?core ans aortirn nhea sisuiea hecgweira

iteration 4000, loss: 68.731909
c.g lrds taptreederereoQrren"utr;n

iteration 0, loss: 45.131108
pped yee corithen a did wowh. Vasind that whing to gene ryant, ald and cbecing to his ftelugh thear wasse, wad under she msleaghr" had they heving, the way noused nit have I rount on a dimal"
TAy lark

iteration 1000, loss: 46.688709
ting as they dad not she way she cuiff curexit, when was himserne bo there simfrot chiofplly he was so the terr to reess bsing was llet orme the then seech in bling a)se thing to bry pismecing and noc

iteration 2000, loss: 45.285481
n toor that whates t haid no wothing, anl ms bere, beby a morith.
"This that thenedy, aby to her thrig that whou'd, s's smaclesk, chatf as, he had nokes beth shim lyon hadkey hendeas, cander aud the d

iteration 3000, loss: 59.130262
t darn in."Shis jurtar thene his marily him siof himsto homa tertound him the anlemed the brecl, sief, therl hord athes dos even buck able forrlasingarst abreathing the dict dight Iwar, aly, moonce, t

iteration 4000, loss: 27.628305
r his wouth it fanstorope and orch 

iteration 0, loss: 39.426882
n tige marither, wece sammed and bel - kicatly could bad.

Wertush, chising down of hem. Had felm ganat suckemer toway even to get ores;.

Wstod br. Sacmeger what they had steak becomane it whele thei

iteration 1000, loss: 35.448479
ng to gef leep an the sidn
 Gregor sppakt hiv sare he cames to her fanding, the clising wot there ffor hid to kte to dawible pace. Him lead of rmook hime no over in is, and to frare no and to cone cre

iteration 2000, loss: 41.759690
ir, peamily its peened up a littre sherngough as Gregor was forhin of hou;d to be oree away ele talk to even be it awly it, ever the though he without urwarss. Unto freal any coomse and becaminat tone

iteration 3000, loss: 59.340685
sn virushidl, kith atwend Gret was the wlolk full to game an the room with that he wazled his stinled wert solely able to hame by wlenged outly on hereving nocailed allbant bach paretly and littlencou

iteration 4000, loss: 24.090868
ss; thenived ank atterted a reishl 

iteration 0, loss: 35.923826
ste lid they had pering ittee always that was back inticed Mr. Samss and now of her hand. Waspitales. This face they and her was not canting that wally had been leanh? Ywing of the room with atthes do

iteration 1000, loss: 36.782055
past of repponely of then the keecl. The dour hard alang mare Gregod buttern gever any difte herse gassion this mother to this cursiouth in, puenthing from three t wnee a lut the juct sset no race on 

iteration 2000, loss: 48.062992
irigg. Buadisbabowas would be very had to be anarmine one of the did could been slletsed Greger was to sharg hore last as if and comly travily who hoplectly orven soon, ant dention. It was butchep, an

iteration 3000, loss: 53.924282
tim nouth round unleave him all beluiregt soon but stice, dent shout,; on the other would of thelf worhad his father hime was buintwn to fall it if his somexing ensiwings was sticule replenedioutly th

iteration 4000, loss: 21.660692
rs and its knaskees to cranl altite

iteration 0, loss: 32.176528
ntoy forn fimm, auread acd oft once they were down in it would have could floch.

Thai'", hadc they das foon misiominally as they kere dooussellese. II they were, "Co herried to distwnew the comfrete 

iteration 1000, loss: 29.522063
lf that the owhe feep now ye prispening wourded hiche we's nnasm weer all, awtice onll anton ffo rant flowly a shizng they the gef" uchaty the key a wasps, heves turning, there a swing pushon. Gregort

iteration 2000, loss: 46.133071
rwease to move but in ole gracked to be at out proce any would sometimes awkit was abread in my slwere ovem vearefully, avmeaby in foll inte vill, it noires.

NI I of his botce in as he doo'makly.

To

iteration 3000, loss: 56.914845
diy hone had liken lout, he called out couth were loying; in his nowo the riothong that she shoutuneded simply infort were sy thong. Now all arare his fakher's was gour to been upliged it withoug and 

iteration 4000, loss: 18.712672
l everyone had uned ore after anste

...

iteration 0, loss: 35.198692
nd ormertuance ang stuge even her sears. Seadr as if she gent than in mest got to get his ant regentes geriabce he could with it as if theig and more believing with their letrurely arring how everyone

iteration 1000, loss: 35.672083
fgs look, think with him, with a resp to reare, now the trand whit there te the offorts think, on the door, "when the regd and here walling as if thore gottroven and ifpress, the chief clerk was suppr

iteration 2000, loss: 42.704054
ice and ontuont, mearly ot mor fornis he had become"edable there earyiefrie concuriaching himner, so much time room from them on soon her hands come in tek nothing. Gregor was prevor had likn on him. 

iteration 3000, loss: 57.601067
n cellens but harre; his way it earlied as have gorently by betam and a gade bome comole; why had been his chanwe both which undortune all omet a turnction;, aboutl to have been probabcention day,. Th

iteration 4000, loss: 18.103721
ted wrang op sometimen trievt. Ohat

iteration 0, loss: 35.172975
sked up onto the benished him now prosen with left bround negeran spunning they they being selet, as if she had terted now"" And fore in versits was that the flat then then distance lou used then at n

iteration 1000, loss: 51.086549
bly cast slikh, said the way seemer mrispian went, it mave, the coo he recomp coom behn cook beforew himself. He ruably though and re leas, now very coppod to see thry wares cookes frot the gevtosn -w

iteration 2000, loss: 52.400653
rtomd all all the kay, Gregor's room-: he doungs, hrawging him ems. She she hom to her.

Te he, once even the lived he had to percered presced that he fust altoo hirried, "ne he would kove mored oukly

iteration 3000, loss: 54.387034
in, buinghile herwerding himself; still from but in a tiosted that mave byen up at him sow hole when he could not which his ranec ut opention by thetes; Mretsed the header it, be heablered from there:

iteration 4000, loss: 22.947162
bie of their kosionided him. Areath

iteration 0, loss: 39.526893
nd cer to te nhe with left t on come if the jubtim one mure albove. I her tod trrions bromm from the timn of be. wherin would fall into the rest gropg with as nems to Greventore the eofintectice in th

iteration 1000, loss: 43.313967
nd again, cale, mare immortathin sursuce,, prine ploormss ughed. Heck, naimh with a lloked rould no,diblt, have like then?, just respiotly whet mishong, as if t in the kighttorg, propably dreauped as 

iteration 2000, loss: 46.763005
rromily up itrued come aroung un contress and even bo berown abatame their mound -wayk.

Mo there flail he huid him ertoor where withirusg come fheck houled himself.

"she'; food, hisgure or etpenffou

iteration 3000, loss: 47.255504
tienchim and rare  arent ur could hat enturied rewm sorthing howe ther could low he could be takeny break fur, mush his hoint now reelied hill of.o lo preis made where ther ha way,s; bread Gregor want

iteration 4000, loss: 23.509213
rntward. Gregor's righ. "'t theard.

iteration 0, loss: 34.011149
gqueccuate sat a long. Wes oun bent doonst of forny we tulp nitht of then, behn donet in a sereat out they were beminct screamed qonathon morn could sqow what to from heray. An. TWs. to the gind or fa

iteration 1000, loss: 36.399725
wv, that meqings and was possible with the othorss the dect, "uribd and whilf with them in their innd with the key? And be secticing.

He really nocking at the wall full furing there quirtle fulling w

iteration 2000, loss: 43.414618
nge in his footherso go them simpettniski" he had movemes for wish if sereared him to cutse his foom -rond vony itto She lithsso even long of his fist. "W he mould remaintlm should hurried to herself 

iteration 3000, loss: 56.348345
lling in a ways; wake the door on her that she kuft in his chise close toway! or straight by tien his wiff was even the time out from , oreered at than it with his waye him his two nowed be near alrou

iteration 4000, loss: 20.708853
nr. Oreftre. Tregorb said yittanded

KeyboardInterrupt: 

Run the following cell to produce sentences using the trained RNN:

In [ ]:
# kafka, including upper chars
for i in range(10):
    print('------ Sentence %i: ------' %i)
    print(rnn.sample('a', 200, char_to_onehot, chars))

Here are some generated sentences for reference how good the results should be. Not quite passing any spell checking program, but there are some real english words in there and the sentences seem to have a reasonable structure.

    ------ Sentence 0: ------
    ted eat aread noup, seed, but. ythed whithit tued that and in ask chemr tirme ble serpareing. he wionly memind. Dedteren drager, chat ther se stealeay. The kpay shir in wha dangereag he torearr ieve w
    ------ Sentence 1: ------
    swayly wase Mlamer io beell hom nofrreang, hr meed tould, Grere, werned coull gitr theQr coule verustengs at jeas whear', fasmer's, ad ho mofello goin "hey in normablite chas a) he. Heenew "thel seefr
    ------ Sentence 2: ------
    r's avethen harly dsundiigex; thinew, oug lertinestad to the wourd jhen sorywe, thever shene It, thour" hawss He:thon's batiar's sfin, luike he. Ar uver. Af. qanilew ucheed attricgwaynong roked t- an 
    ------ Sentence 3: ------
    tor
     unde, thab haw dtidtly kad sendeder, ithed far theme, veedlly nowlyin; and )rece thete disse, out louthe, row the rapeasterse at. Heed rnweem. "ut he bleitny Grerorot ie reryibe. So in., Hers ane
    ------ Sentence 4: ------
    ding, fitse. . Whisly andit, she haing "ya bistatily dlatkedor whing saded theust, but the stere stotady is eleaprcentake, got to mame luch ally, mus llablly, ach caling, at what, he want aar athen d'
    ------ Sentence 5: ------
    cll"end sat what herpingoth thouthar ald beinvaver.

    Tere sreaned on in she was athar thatssu fres is at lees morther all sroaven to somichary mothtrey norytiot a tior sow bercead lassedore towist at 
    ------ Sentence 6: ------
    gly.

    Shen in shend comrie t oreth ubserwer he dist in commicherr. Wrmeathithemy nougracl.y'e sate horkllomisted betrnowhrous wfverthr bleatekeM tusinitedy Herach mop wislrocly theirse
     on illy ans in
    ------ Sentence 7: ------
    dd, puting ot been ay theed . Hemula, he farer". The h hanor, and dandid to the sedertereveng untint.

    Wken", surme ceared thet lomp io wotcouUir onsur"ont ar with, thancMmperny him., is sas bees coul
    ------ Sentence 8: ------
    d natmen", he theretotist wist, Gregely, arly re caule he praser, and woll buccem, then atse. Hemu haded ditince and dleyrist comply. "Vm inen beglings wet ou't dot, plest willay theren lipsest inpren
    ------ Sentence 9: ------
    byed Gregst oonsbecemed thet ad herladble, din ther, and at. Numferringtice st ead thelr way mors. Ium loomul themo head roon inf thet the. "lencorn was ned in tuc'e foot butimed, -feritn be thand Yun

You may try different learning rates, different number of neurons for the hidden state and try to prune the text from special characters and upper characters, but the fact is that this is only one single RNN cell and it is as good as it gets.  

## 2 Neural Machine Translation <a class="anchor" id="2"></a>

In this task you will implement a small neural machine translator using the Keras API. The final model will be able to translate english sentences into spanish sentences. Unlike the previous task, the input and output at each time step will now represent an entire word instead of a character.  

The task is divided into two main sections:
 * Pre processing of data
 * Building a sequence-to-sequence RNN
 
The principle of sequence-to-sequence modelling is shown in the figure below
![seq2seq unrolled](utils/images/seq2seq.png)
In your case, an english sentence is input to the encoder network, one word at each time step. After the sentence has been fed, there will now be a representation of the entire sentence encoded into the hidden state of the RNN. The hidden state of the decoder network is thereafter initialized with this representation and tries to generate the sequence of spanish words from the initial hidden state.  

Run the cell below to import the necessary packages to get started with the task

In [1]:
import re
import numpy as np
import unidecode
from keras.preprocessing.sequence import pad_sequences
from utils.tests.ha2Tests import *

D:\Python\Anaconda\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### 2.1 Pre-processing <a class="anchor" id="2.1"></a>
The dataset to use consists of bilingual sentence pairs. Each sentence in the list of english sentences has a corresponding spanish translation at the same index in the list of spanish sentences.  

Basic preprocessing has already been performed such as:  
 * to lower case
 * remove any padded spaces at the start and end of the row (trim)
 * convert from unicode character set to the ASCII character set. Example: á --> a and ñ --> n.
 * If the character is a question mark, punctuation or exclamation mark: ? ! ., put a space to the left of the character to make the char into a separate word.
 * remove any non-letter character
 * split each line into a tuple, where the left element is the source language sentence and the right element is the target language sentence
 * remove any sentence pair with number of words more than 5

#### 2.1.1 Loading and inspecting the data set <a class="anchor" id="2.1.1"></a>
Lets first load the english-spanish sentence pairs into memory and inspect some samples.

In [2]:
# load data set
lines = open('utils/spa.txt', 'r', encoding='UTF-8').read().strip().split('\n')
data = pickle.load( open( "utils/spa-preprocessed.pkl", "rb" ) )
spa, eng = data['spa'], data['eng']
SEQ_MAX_LEN = 5

print('There are %i sentence pairs' % len(lines))
print('The shortest english sentence is %i words' 
      % np.min(list(map(lambda line: len(line.split(' ')), eng))))
print('The shortest spanish sentence is %i words' 
      % np.min(list(map(lambda line: len(line.split(' ')), spa))))
print('The longest english sentence is %i words' 
      % np.max(list(map(lambda line: len(line.split(' ')), eng))))
print('The longest spanish sentence is %i words' 
      % np.max(list(map(lambda line: len(line.split(' ')), spa))))
print('\n10 Random short sentence pairs:')
for i in range(10):
    ix = np.random.choice(int(len(eng)))
    print(eng[ix],'\t',spa[ix])

There are 114282 sentence pairs
The shortest english sentence is 2 words
The shortest spanish sentence is 2 words
The longest english sentence is 5 words
The longest spanish sentence is 5 words

10 Random short sentence pairs:
tom was hospitalized . 	 tom fue hospitalizado .
he ruined my life . 	 el arruino mi vida .
im grateful . 	 soy agradecida .
dont sing . 	 no canten .
they found him guilty . 	 lo encontraron culpable .
i like your frankness . 	 me gusta su franqueza .
were not in boston . 	 no estamos en boston .
no one knew it . 	 nadie lo sabia .
its not just that . 	 no solo es eso .
dont leave them alone . 	 no los deje solos .


#### 2.1.4 Word-to-index and index-to-word conversions <a class="anchor" id="2.1.4"></a>
Just like in Task 1 earlier, you need dictionaries to transform between (in this case) the words and unique number representations. The following dictionaries/lists are implemented:  

 * `eng_ix_to_word` - a python list where each index is a unique number that represents the english word stored as value.
                      `eng_ix_to_word[0]` should equal to `ZERO`
                      `enx_ix_to_word[-1]` (the last element) should equal to `UNK` (unknown)
 * `eng_word_to_ix` - like `eng_ix_to_word` but reversed, a dictionary mapping every word into the unique number. The key is a word that also exists in `eng_ix_to_word` and the value is an integer, that matches the index of the word in `eng_ix_to_word`
 * `spa_ix_to_word` - same principle as `eng_ix_to_word`, but from the vocabulary of `spa`
                      `spa_ix_to_word[0]` should equal to `ZERO`
                      `spa_ix_to_word[-1]` (the last element) should equal to `UNK` (unknown)
 * `spa_word_to_ix` - same principle as `eng_word_to_ix`, but from the vocabulary of `spa`

In [3]:
eng_vocab = set([word for sentence in eng for word in sentence.split(' ')])
eng_ix_to_word = ['ZERO'] + list(eng_vocab) + ['UNK']
eng_word_to_ix =  { word: index for index, word in enumerate(eng_ix_to_word)}
spa_vocab = set([word for sentence in spa for word in sentence.split(' ')])
spa_ix_to_word = ['ZERO'] + list(spa_vocab) + ['UNK']
spa_word_to_ix =  { word: index for index, word in enumerate(spa_ix_to_word)}

Now we convert the words of `eng` and `spa` into their corresponding index numbers  

Run the cell below to initialize these variables:  
 * X - A list of sentences. Each sentence is represented by a list where the i'th element is the index number representing the i'th word in the sentence string.
 * Y - same principle as `X`, but contains the converted spanish sentences  
 
 Example:
 if `eng_word_to_ix` is defined as `{'good':0, 'morning':1}`  
 and `eng` is defined as `["good morning", "morning"]`  
 then `X` should result in `[[0,1],[1]]`  

In [4]:
X = [ [eng_word_to_ix[word] for word in sentence.split(' ')] for sentence in eng]
Y = [ [spa_word_to_ix[word] for word in sentence.split(' ')] for sentence in spa]

#### 2.1.5 Padding <a class="anchor" id="2.1.5"></a>
Since the sentences are of variable lengths, padding is needed to make every sentence equal length. Every sentence list in `X` and `Y` are padded with zeros to make every list equal to the length of the maximum sentence length. The zeros are **prepended** before the first word.  

Example:  
if `X` is defined as `[[1,2,3], [1,2], [1]]`   
then after applying padding, X is equal to `[[1,2,3], [0,1,2], [0,0,1]]`  

In [5]:
X = pad_sequences(X, maxlen=SEQ_MAX_LEN, dtype='int32')
Y = pad_sequences(Y, maxlen=SEQ_MAX_LEN, dtype='int32')

#### 2.1.6 One-hot labels <a class="anchor" id="2.1.6"></a>
The last pre processing converting the data into one-hot vectors.  

For this task, **only** the labels (spanish words) are going to be transformed into one-hot vectors. The data points of english words are only going to be transformed into their corresponding numeric values by `eng_word_to_ix` because `Embedding` from the keras API already implements converting the inputs to one-hot vectors.  

Transform `Y` to have the following properties:
 * define `Y` as a numpy matrix of shape (number of sentences, number of words of longest sentence, one-hot vector length). 
 * dimension 0 represents each sentence (batch).
 * dimension 1 represents each word in a sentece (padded with zeros for equal length)
 * dimension 2 represents the one-hot vector for that particular word in that particular sentence. To pass the test, a onehot representation of word `1023` should be a zero vector with its 1023th index set to 1

In [6]:
# TODO: convert each index number in Y into its corresponding one-hot vector
num_output = len(spa_word_to_ix)
print("Tip: the shape of Y should be: (%i, %i, %i)" % (len(Y), len(Y[0]), num_output))

def Y_to_onehot(Y, num_output):
    # Complete function according to description above
    # YOUR CODE HERE

    onehot1 = np.zeros([len(Y), len(Y[0]), num_output])
    for i in range(len(Y)):
        for j in range(len(Y[0])):
            one = Y[i,j]
            onehot1[i,j,one] = 1
    return onehot1
# def get_one_hot(targets, nb_classes):
#     res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
#     return res.reshape(list(targets.shape)+[nb_classes])        

# onehot = np.zeros((len(Y), len(Y[0]), num_output))
# print(onehot)
# m = np.matrix([[[1,2],[3,4]],[1,2],[3,4]])       
# m[0,0][0]

Tip: the shape of Y should be: (22947, 5, 8805)


Run the following cell to test your implementation.

In [7]:
# test case 
test_Y_to_onehot(Y_to_onehot)

test passed


When you passed the test, run the cell below to convert the sentences of Y into lists of onehot vectors

In [8]:
Y_onehot = Y_to_onehot(Y, num_output)

### 2.2 Implementing a sequence-to-sequence model <a class="anchor" id="2.2"></a>
In this section you will define, train and evaluate a sequence-to-sequence RNN architecture with the help of the Keras API.  

#### 2.2.1 Defining the architecture <a class="anchor" id="2.2.1"></a>

Now you are going to define the architecture for the translator. There are several possible architectures for performing translation, which is a sequence-to-sequence task (an input sequence, the English sentence, mapped to an output sequence, the Spanish sentence). If you take inspiration from searching the internet, you might come across neural machine translators such as [Google's Neural Machine Translation System](http://tsong.me/blog/google-nmt/). This kind of deep architecture was trained for a week using 100 GPUs, which is completely out of reach for the purposes of this assignment. Instead we will ask you to implement a simpler architecture that, even though it doesn't possess all the important parts of a translation network, still performs acceptably.

The architecture you are required to implement will closely resemble the image shown at the start of this task. First the input sequence is fed to an encoder network, which is in charge of summarizing the entire sequence in its internal memory. This internal memory is then fed to a decoder network, which uses this summarized content to produce the output sequence. Both the encoder and the decoder networks will be comprised of a single LSTM unit, with a memory cell of arbitrary dimension. Since we won't stack LSTM layers, this can be seen as a "toy" version of a complete LSTM architecure.

#### Practical remarks

Note that the input sequence for each training sentence is a vector with 5 elements, where each element corresponds to the number of that particular word in the vocabulary. 

Because similar word indexes don't correspond to similar words (e.g. index 133 corresponds to 'joke', whereas index 134 corresponds to 'silence'), it's easier to train our network if we map this data to a space where the semantic meaning of the word is related to its geometric position. For instance, we would like the word 'car' to be closer to the word 'bus' than it is to 'tree'. This can be accomplished with the `Embedding` layer module from Keras, which tries to find a suitable representation for positive integers in a higher-dimensional space by using dimensionality reduction techniques in the matrix of co-ocurrence statistics of our input data. This layer can use an already trained mapping (like Word2Vec), but in this exercise we'll train it ourselves.

Besides deciding how to encode the input, there is another important practical remark to be done. Since the decoder network will also be implemented as an LSTM, it will require inputs. According to what was described earlier, the only important data for this network is the memory cell at each time-step. Unfortunately, there is no simple way to create an LSTM cell without inputs (you can, of course, customize the LSTM layer to not receive any inputs), so we'll simply feed zeros as input at each time-step and it's likely that the optimization procedure will tune the decoder weights to disregard this input eventually (because it's completely uncorrelated to the input and the label, so it doesn't help with the prediction at all).

#### Keras tips

- It's necessary to use the [Model class API](https://keras.io/models/model/) for implementing this model (because it has two inputs, the input sequence and the zeros to the decoder). Be sure to familiarize yourself with this way of defining models in Keras.
- You can define an LSTM cell using the [LSTM layer](https://keras.io/layers/recurrent/#lstm).
- The LSTM layer by default outputs only the output for the last time-step, which is great for the encoder part, since we don't care about its outputs anyway. However, for the decoder network we want the outputs at all time steps. To accomplish this, you can set the argument `return_sequences` to `True` when creating the LSTM layer.
- You will need to feed the memory cell content of the encoder network at the last time-step to the decoder network. You can ask the LSTM layer to also output its memory cell contents by setting the `return_state` argument to `True` when creating the layer (this makes the LSTM layer output three things: the actual LSTM output and two variables with different parts of the LSTM internal memory cell).
- You can specify the initial state of LSTM layers symbolically by calling them with the keyword argument  `initial_state`. The value of `initial_state` should be a tensor or list of tensors representing the initial state of the LSTM layer.


Now, implement your sequence-to-sequence architecture in the function `create_model` by following the descriptions of the function.  

In [10]:
# Add your import statements here
from keras import Input, Model
from keras.layers import Activation, TimeDistributed, Dense, RepeatVector, Embedding
from keras.layers.recurrent import LSTM
from keras.datasets import mnist
from keras.layers import Dense, LSTM
from keras.utils import to_categorical
from keras.models import Sequential
import spacy


def create_model(input_n, X_seq_len, output_n, Y_seq_len, hidden_dim, embedding_dim):
    """ Define a keras sequence-to-sequence model. 
    
    Arguments:
    input_n - integer, the number of inputs for the network (the length of a one-hot vector from `X`)
    X_seq_len - integer, the length of a sequence from `X`. Should be constant and you made sure by using padding
    output_n - integer, the number of outputs for the network (the length of a one-hot vector from `Y`)
    Y_seq_len - integer, the length of a sequence from `Y`. Should be constant and you made sure by using padding
    hidden_dim - integer, number of units in the LSTM's memory cell.
    embedding_dim - output dimension of the embedding layer.
    
    Returns:
    The compiled keras model
    
    """
    # Input and embedding layers
    encoder_input_layer = Input(shape=[X_seq_len])
    #print(Input(shape=[X_seq_len]))
    encoder_embedding_layer = Embedding(input_n, embedding_dim, input_length=X_seq_len, mask_zero=True)(encoder_input_layer)
    
    # Create the encoder LSTM.
    # YOUR CODE HERE
    encoder_LSTM1 = LSTM(units=hidden_dim,
                         return_state=True)
    encoder_output,hidden_state,cell_state = encoder_LSTM1(encoder_embedding_layer)
    encoder_state = [hidden_state,cell_state]
#     encoder_LSTM2 = LSTM(units=hidden_dim,
#                                      return_state=True,
#                          input_shape=(X_seq_len,input_n),
#                                      kernel_initializer='glorot_uniform')(encoder_LSTM1)
#     encoder_LSTM3 = LSTM(units=hidden_dim,
#                                      return_state=True,
#                          input_shape=(X_seq_len,input_n),
#                                      kernel_initializer='glorot_uniform')(encoder_LSTM2)
#     encoder_LSTM4 = LSTM(units=hidden_dim,
#                                      return_state=True,
#                          input_shape=(X_seq_len,input_n),
#                                      kernel_initializer='glorot_uniform')(encoder_LSTM3)
#     encoder_LSTM5 = LSTM(units=hidden_dim,
#                                      return_state=True,
#                          input_shape=(X_seq_len,input_n),
#                                      kernel_initializer='glorot_uniform')(encoder_LSTM4)
                    
    
                                         
                                    # input_shape=(X_seq_len,input_n),
    # Null input (should be fed with zeros) and repeating it the same number of times as there are words in the 
    # target sentence
    null_input = Input(shape=[1])
    repeated_null = RepeatVector(Y_seq_len)(null_input)
    
    # Create the decoder LSTM (feed it with the memory of the encoder network). Call it `decoder_layer`
    # YOUR CODE HERE
    decoder_LSTM = LSTM(units=hidden_dim,
                        return_state=True,
                        return_sequences=True)
    decoder_layer,hidden_layer1,cell_state1 = decoder_LSTM(repeated_null,initial_state=encoder_state)
    
    # Add a fully connected layer and a softmax to the outputs of the decoder
    decoder_fully_connected = TimeDistributed(Dense(output_n))(decoder_layer)
    decoder_softmax = Activation('softmax')(decoder_fully_connected)
    
    # Create final model and compile it
    model = Model([encoder_input_layer, null_input], decoder_softmax)
    
    # Compile the model. Use a loss function, optimizer, and metrics of your choice
    # YOUR CODE HERE
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    
    
    
    # Add these arguments to the model for convenience
    model.hidden_dim = hidden_dim
    model.embedding_dim = embedding_dim
    
    return model

#### 2.2.2 Training the model <a class="anchor" id="2.2.2"></a>
Now create the model and train it. Try out different hyper-parameters if you're not satisfied with the result. For obtaining a good speedup using the GPU, opt for large number of memory cells and large batch sizes (although not too large, that also has drawbacks).

In [11]:
# Computing inputs for `create_model`
input_n = len(eng_word_to_ix)
output_n = len(spa_word_to_ix)
X_seq_len = len(X[0])
Y_seq_len = Y_onehot.shape[1]

Xz = np.zeros([22947,1])
# Create the model
# YOUR CODE HERE
model = create_model(input_n, X_seq_len, output_n, Y_seq_len, 512, 512)

# Train the model (note that you should feed both the input sentence and a zero array with the correct shape to
# the `fit` method).
# YOUR CODE HERE
#model.fit([X,Xz], Y_onehot,epochs=20, batch_size=128, validation_split=0.2, verbose=2)

#model.save_weights('seq2seq_model_correct.hdf5')

# If you need, you can use the following line for loading a saved model weights
#model.load_weights('seq2seq_model.hdf5')

#### 2.2.3 Evaluation <a class="anchor" id="2.2.3"></a>
You are free to evaluate and compare results of your model(s) in any way you have learned from the deep learning course.  

Now motivate your choice of architecture and hyperparameters by answering the questions below.

**Question:** What loss function, metrics and optimizer did you use and why?

**Your answer:** (fill in here)

**Question:** Did you use any Keras callbacks? If so, how did they help you?

**Your answer:** (fill in here)

**Question:** How did you evaluate that the model was good enough?

**Your answer:** (Once the model converges to a minimum point, compare the loss gap between the training data and validation data, if it is )

#### 2.2.4 Testing <a class="anchor" id="2.2.4"></a>
Test your neural machine translator on a few sentences by running the test case below. The similarity metric is calculated comparing word embeddings.  

You can also use the function `translate` to try your own sentences, just remember that every word needs to be in the vocabulary!

In [12]:
model = model.load_weights('seq2seq_model_correct.hdf5')
def translate(sentence):
    """ Translate a sentence using `model`. `model` is assumed to be a global variable.
    
    Arguments:
    sentence - a string to translate
    
    Returns:
    the translated sentence as a string
    """
    
    # Check that each of the words in the input sentence exist in the input dictionary
    try:
        x = [eng_word_to_ix[word] for word in sentence.split(' ')]
    except KeyError as e: 
        print('{0} doesn\'t exist in the vocabulary!'.format(e))
        
    # Pad the input sentence
    x = pad_sequences([x], maxlen=SEQ_MAX_LEN, dtype='int32')
    
    # YOUR CODE HERE
    
    y_pred_words = ""
    y_pred_probs = model.predict([x,np.zeros(len(x))])
    for i in range(len(y_pred_probs)):
        for j in range(len(y_pred_probs[i])):
            ix = np.argmax(y_pred_probs[i,j])
            if ix == 0:
                continue
            w = spa_ix_to_word[ix]
            y_pred_words += " " + w
    return y_pred_words


In [13]:
# test case. be sure the variable model has a reference to your trained model
test_predictions(translate, eng_word_to_ix, spa_ix_to_word, SEQ_MAX_LEN, model)

OSError: [E050] Can't find model 'es'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [ ]:
# test for yourself
print(translate('this is a dog .'))

In [ ]:
print(translate('she love apples .'))

In [ ]:
print(translate('he opened the window .'))

In [ ]:
print(translate('this is working well .'))

In [ ]:
print(translate('i am a great student .'))

## Congratulations!
You have successfully implemented a recurrent neural network from scratch using only NumPy and also implemented a neural machine translator using Keras!

When choosing the architecture for your neural machine translator you are partly restricted from the capabilities of Keras and partly restricted from your available computing power.  

**Question:** Give 3 suggestions for different techniques that can be used to improve your neural machine translator.

**Your answer:** (fill in here)